In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from pydrake.all import MosekSolver

from spp.bezier import BezierSPP

In [ ]:
def build_bezier_gcs(regions):
    order = 7
    continuity = 4
    vel_limit = 10 * np.ones(3)
    hdot_min = 1e-3
    weights = {"time": 1., "norm": 1.}
    
    gcs = BezierSPP(regions, order, continuity, hdot_min=hdot_min)
    gcs.addTimeCost(weights["time"])
    gcs.addPathLengthCost(weights["norm"])
    gcs.addVelocityLimits(-vel_limit, vel_limit)
    gcs.setPaperSolverOptions()
    gcs.setSolver(MosekSolver())

    return gcs

In [ ]:
start = np.array([-1, -1])
goal = np.array([2, 1])
start_pose = np.r_[(start-start)*5, 1.]
goal_pose = np.r_[(goal-start)*5., 1.]

zero_deriv_boundary = 3

region_file = "room_debug/regions.reg"
with open(region_file, "rb") as f:
    regions = pickle.load(f)
    
np.random.seed(0)
relax_gcs = build_bezier_gcs(regions)
relax_output = relax_gcs.SolvePath(start_pose, goal_pose, True, False,
                                   zero_deriv_boundary=zero_deriv_boundary, preprocessing=True)
b_traj_relax, result_relax, best_result_relax, hard_result_relax, _ = relax_output